# Sentiment analysis of Formula 1 radio messages

This is the second notebook of the task of analyzing Formula 1 radios and extracting valuable information.

For the sentiment analysis, first, we will use **nltk** with Vader. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from tqdm.notebook import tqdm

# Set styling for visualizations
plt.style.use('ggplot')
sns.set(style="whitegrid")